In [1]:
import numpy as np 

In [3]:
m = np.arange(0, 25).reshape((5, 5))

out = np.triu(m)
inp = np.tril(m)
out, inp

(array([[ 0,  1,  2,  3,  4],
        [ 0,  6,  7,  8,  9],
        [ 0,  0, 12, 13, 14],
        [ 0,  0,  0, 18, 19],
        [ 0,  0,  0,  0, 24]]),
 array([[ 0,  0,  0,  0,  0],
        [ 5,  6,  0,  0,  0],
        [10, 11, 12,  0,  0],
        [15, 16, 17, 18,  0],
        [20, 21, 22, 23, 24]]))

In [4]:
adj = np.random.randint(0, 2, size=(5, 5))
adj

array([[1, 0, 1, 0, 0],
       [1, 0, 0, 1, 1],
       [1, 0, 0, 1, 0],
       [1, 1, 1, 0, 0],
       [1, 0, 1, 1, 1]])

In [12]:
((inp + out) * adj)

array([[ 0,  0,  2,  0,  0],
       [ 5,  0,  0,  8,  9],
       [10,  0,  0, 13,  0],
       [15, 16, 17,  0,  0],
       [20,  0, 22, 23, 48]])

In [13]:
import torch

In [24]:
a = torch.tensor([1, 2, 3])
a.unsqueeze_(-1)

tensor([[1],
        [2],
        [3]])

In [29]:
r = torch.tensor(((inp + out) * adj))
r.unsqueeze_(0)
print(r.shape)
r[r > 0].flatten(start_dim=0, end_dim=1)

torch.Size([1, 5, 5])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [35]:
r = r.repeat((2, 1, 1))

In [41]:
r = torch.flatten(r, start_dim=1, end_dim=-1)
r[r>0]

tensor([[ 0,  0,  2,  0,  0,  5,  0,  0,  8,  9, 10,  0,  0, 13,  0, 15, 16, 17,
          0,  0, 20,  0, 22, 23, 48],
        [ 0,  0,  2,  0,  0,  5,  0,  0,  8,  9, 10,  0,  0, 13,  0, 15, 16, 17,
          0,  0, 20,  0, 22, 23, 48]], dtype=torch.int32)

In [42]:
r[r > 0].reshape(2, -1)

tensor([[ 2,  5,  8,  9, 10, 13, 15, 16, 17, 20, 22, 23, 48],
        [ 2,  5,  8,  9, 10, 13, 15, 16, 17, 20, 22, 23, 48]],
       dtype=torch.int32)

In [53]:
import torch.nn as nn

In [90]:
class GraphGrad(torch.nn.Module):
    def __init__(self, adj_mx):
        """Graph gradient operator that transform functions on nodes to functions on edges.
        """
        super(GraphGrad, self).__init__()
        self.adj_mx = adj_mx
        self.grad = self._grad(adj_mx)
    
    @staticmethod
    def _grad(adj_mx):
        """Fetch the graph gradient operator."""
        num_nodes = adj_mx.size()[-1]

        num_edges = (adj_mx > 0.).sum()
        grad = torch.zeros(num_nodes, num_edges)
        e = 0
        for i in range(num_nodes):
            for j in range(num_nodes):
                if adj_mx[i, j] == 0:
                    continue

                grad[i, e] = 1.
                grad[j, e] = -1.
                e += 1
        return grad

    def forward(self, z):
        """Transform z with shape `(..., num_nodes)` to f with shape `(..., num_edges)`.
        """
        return torch.matmul(z, self.grad)

In [68]:
adj

array([[1, 0, 1, 0, 0],
       [1, 0, 0, 1, 1],
       [1, 0, 0, 1, 0],
       [1, 1, 1, 0, 0],
       [1, 0, 1, 1, 1]])

In [84]:
gg = GraphGrad(torch.tensor(adj))
grad = gg.grad
grad.shape

torch.Size([5, 14])

In [94]:
grad.transpose(-1, -2).shape

torch.Size([14, 5])

In [97]:
grad.size(-1)

14

In [73]:
inp.shape

(5, 5)

In [88]:
gg(torch.tensor(inp, dtype=torch.float32))

tensor([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.],
        [ -5.,   5.,   1.,   6.,   6.,  -5.,   0.,  -5.,  -6.,   0.,  -5.,   0.,
           0.,   0.],
        [-10.,  -2.,   1.,  11.,  11.,   2.,  12., -10., -11., -12., -10., -12.,
           0.,   0.],
        [-15.,  -2.,   1.,  -2.,  16.,   2.,  -1.,   3.,   2.,   1., -15., -17.,
         -18.,   0.],
        [-20.,  -2.,   1.,  -2.,  -3.,   2.,  -1.,   3.,   2.,   1.,   4.,   2.,
           1., -24.]])

In [80]:
torch.tensor(inp).shape 
# (grad_T.T)

torch.Size([5, 5])

In [81]:
torch.matmul?

Docstring:
matmul(input, other, *, out=None) -> Tensor

Matrix product of two tensors.

The behavior depends on the dimensionality of the tensors as follows:

- If both tensors are 1-dimensional, the dot product (scalar) is returned.
- If both arguments are 2-dimensional, the matrix-matrix product is returned.
- If the first argument is 1-dimensional and the second argument is 2-dimensional,
  a 1 is prepended to its dimension for the purpose of the matrix multiply.
  After the matrix multiply, the prepended dimension is removed.
- If the first argument is 2-dimensional and the second argument is 1-dimensional,
  the matrix-vector product is returned.
- If both arguments are at least 1-dimensional and at least one argument is
  N-dimensional (where N > 2), then a batched matrix multiply is returned.  If the first
  argument is 1-dimensional, a 1 is prepended to its dimension for the purpose of the
  batched matrix multiply and removed after.  If the second argument is 1-dimensional, a
